In [1]:
from finetuning_utils import *
import torch.nn as nn

Using cuda as DEVICE


In [2]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
weights_dir = config['data']['model_weights']

### Get Loaders

In [3]:
train_loader, test_loader = get_munich_tuning_loaders(batch_size=2)

Indexing files in orthophotos/labeling_subset... 
Found: 	 150 Images 
	 83 Mask
Length of all files: 83
Length of train (66) and test (17): 83
Length of train loader: 33; Length of test loader: 9 with batch size 2


### Training

In [4]:
 def train_epoch(model, train_loader, epoch):
    """
    Trains the model for one epoch using the provided training data.

    Args:
        model (torch.nn.Module): The model to be trained.
        train_loader (torch.utils.data.DataLoader): The DataLoader object that provides the training data.
        epoch (int): The current epoch number.

    Returns:
        None
    """
    model.train()
    loss_sum = 0
    for batch_id, (data, label) in enumerate(train_loader):
        # Send data and label to DEVICE
        data = data.to(DEVICE)
        label = label.to(DEVICE)

        with torch.cuda.amp.autocast(): 
            # Forward Pass:
            output = model(data)['out']

            # Caluclate Loss
            loss = LOSS_FUNC(output, label.long())

        # Backward Pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Comupte loss sum and count batches
        loss_sum += loss.item()
        batch_id += 1

        if batch_id%5==0:
            progress = f'Epoch: {epoch} | Batch {batch_id} / {len(train_loader)} | Loss: {np.round(loss_sum/(batch_id),4)}'
            print(progress)

            

In [5]:
def train_model(model, train_loader, test_loader, LEARNING_RATE = 0.01, NUM_EPOCHS=1):
    #################### Training Setup #################### 
    # Send model to DEVICE
    model = model.to(DEVICE)    

    # Optimization setup
    global optimizer
    global LOSS_FUNC
    global scaler
    optimizer = optim.SGD(params=model.parameters(), lr=LEARNING_RATE)
    LOSS_FUNC = nn.CrossEntropyLoss()
    # use torch grad scaler to speed up training and make it more stable
    scaler = torch.cuda.amp.GradScaler()

    #################### Training #################### 
    result = calculate_accuracy(model, test_loader, 6)
    display(result)
    print("Start Training ...")
    for epoch in range(NUM_EPOCHS):
        train_epoch(model, train_loader, epoch)
        result = calculate_accuracy(model, test_loader, 6)
        display(result)

        #################### Saving ####################
        # Save model to disk
        #torch.save(model, save_dir + '/' + specs_name+'_epoch'+str(epoch)+'.pth.tar')
        #torch.save(optimizer, save_dir + '/' + specs_name+'_optimizer.pth.tar'


In [7]:
model_name = 'loveda_DeepLabV3_r101_epochs15_lr0-07487'
model = torch.load(weights_dir + '/own_training/' + model_name + '/' + model_name + '_epoch14.pth.tar', 
                map_location=torch.device('cpu'))
model = model.to(DEVICE)

train_model(model, train_loader, test_loader, LEARNING_RATE=0.1, NUM_EPOCHS = 10)


Evaluating Accuracy on Test Set...


100%|██████████| 9/9 [00:01<00:00,  6.58it/s]


,Metric,Accuracy (%)
0,ignore,No instances in the test set
1,impervious,80.5
2,building,61.36
3,low vegetation,52.04
4,water,82.02
5,trees,83.7
6,Mean Accuracy,71.924
7,Average Accuracy,71.219635


Start Training ...
Epoch: 0 | Batch 5 / 33 | Loss: 2.1357
Epoch: 0 | Batch 10 / 33 | Loss: 1.7153
Epoch: 0 | Batch 15 / 33 | Loss: 1.6171
Epoch: 0 | Batch 20 / 33 | Loss: 1.4887
Epoch: 0 | Batch 25 / 33 | Loss: 1.4334
Epoch: 0 | Batch 30 / 33 | Loss: 1.352

Evaluating Accuracy on Test Set...


100%|██████████| 9/9 [00:01<00:00,  6.65it/s]


...

100%|██████████| 9/9 [00:01<00:00,  6.54it/s]


,Metric,Accuracy (%)
0,ignore,No instances in the test set
1,impervious,69.74
2,building,58.34
3,low vegetation,84.36
4,water,11.09
5,trees,91.67
6,Mean Accuracy,63.04
7,Average Accuracy,80.07339


Epoch: 9 | Batch 5 / 33 | Loss: 0.5735
Epoch: 9 | Batch 10 / 33 | Loss: 0.5209
Epoch: 9 | Batch 15 / 33 | Loss: 0.5228
Epoch: 9 | Batch 20 / 33 | Loss: 0.485
Epoch: 9 | Batch 25 / 33 | Loss: 0.4769
Epoch: 9 | Batch 30 / 33 | Loss: 0.4783

Evaluating Accuracy on Test Set...


100%|██████████| 9/9 [00:01<00:00,  6.52it/s]


,Metric,Accuracy (%)
0,ignore,No instances in the test set
1,impervious,66.54
2,building,62.36
3,low vegetation,93.77
4,water,0.21
5,trees,76.0
6,Mean Accuracy,59.776
7,Average Accuracy,76.90856
